In [6]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [8]:
tf.keras.backend.clear_session()

In [9]:
subject = 'Bahia - Consumo de Cimento (t)'
start_index = 0
split_index = 203 #Referente aos 230 anos de input  
train_split = split_index + 1 - 12*3

In [10]:
data = pd.read_csv('2003_mo_model_input_BA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Unnamed: 0,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego,Bahia - Consumo de Cimento (t)
0,2003-1,0.299858,0.724032,-5331.049150,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.669899,39.798880,1.317344e+08,8.384593e+06,8.566149,1.216359e+08,8.348779,151.297
1,2003-2,0.301903,0.690297,-5318.079644,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.670210,39.480034,1.318964e+08,8.391946e+06,8.569210,1.216914e+08,8.342979,138.707
2,2003-3,0.303709,0.669681,-5436.417870,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.670521,39.400256,1.320584e+08,8.399299e+06,8.572270,1.217469e+08,8.337179,135.009
3,2003-4,0.305311,0.660494,-5707.015274,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.670831,39.417185,1.322204e+08,8.406652e+06,8.575331,1.218023e+08,8.331379,126.554
4,2003-5,0.306860,0.648337,-5599.317941,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.671142,39.479943,1.323824e+08,8.414005e+06,8.578392,1.218578e+08,8.325579,137.331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,0.597113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.069163,NaN,NaN,NaN,NaN,NaN,366.305
236,2022-9,0.596178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.752943,NaN,NaN,NaN,NaN,NaN,346.042
237,2022-10,0.594662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.537361,NaN,NaN,NaN,NaN,NaN,347.901
238,2022-11,0.592436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.971241,NaN,NaN,NaN,NaN,NaN,310.845


In [20]:
input_data = data.iloc[:split_index + 1,1:-1]
# input_data = data.iloc[:split_index + 1,:]
input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
input_data

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
0,-1.436732,2.782450,0.097384,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489,-2.195846,-1.793459,-1.766933,-0.695996,-2.315693,-2.238131,-0.944845
1,-1.404185,2.407943,0.100462,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073,-2.161313,-1.806616,-1.747868,-0.654222,-2.273375,-2.196868,-0.946500
2,-1.375442,2.179073,0.072380,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505,-2.126781,-1.809908,-1.728803,-0.612447,-2.231056,-2.155605,-0.948156
3,-1.349941,2.077086,0.008167,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220,-2.092248,-1.809210,-1.709739,-0.570673,-2.188738,-2.114342,-0.949811
4,-1.325290,1.942128,0.033724,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493,-2.057715,-1.806620,-1.690674,-0.528898,-2.146420,-2.073079,-0.951467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.851312,-0.689886,2.909383,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628,1.041952,0.619029,0.865987,-1.416165,0.555618,0.463875,1.107985
200,1.873607,-0.441954,2.913017,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775,1.026783,0.635352,0.848481,-1.404857,0.531641,0.443334,1.106555
201,1.901718,-0.132782,2.818435,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287,1.011615,0.652040,0.830975,-1.393550,0.507664,0.422793,1.105126
202,1.928136,0.084061,2.905726,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156,0.996447,0.638794,0.813469,-1.382243,0.483687,0.402252,1.103697


In [21]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      138.707
1      135.009
2      126.554
3      137.331
4      118.680
        ...   
235    346.042
236    347.901
237    310.845
238    310.845
239        NaN
Name: Bahia - Consumo de Cimento (t), Length: 240, dtype: float64

In [22]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
0,-1.436732,2.782450,0.097384,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489,-2.195846,-1.793459,-1.766933,-0.695996,-2.315693,-2.238131,-0.944845
1,-1.404185,2.407943,0.100462,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073,-2.161313,-1.806616,-1.747868,-0.654222,-2.273375,-2.196868,-0.946500
2,-1.375442,2.179073,0.072380,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505,-2.126781,-1.809908,-1.728803,-0.612447,-2.231056,-2.155605,-0.948156
3,-1.349941,2.077086,0.008167,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220,-2.092248,-1.809210,-1.709739,-0.570673,-2.188738,-2.114342,-0.949811
4,-1.325290,1.942128,0.033724,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493,-2.057715,-1.806620,-1.690674,-0.528898,-2.146420,-2.073079,-0.951467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.005306,-0.742853,0.615006,-0.619453,0.617009,-0.970725,0.980624,0.764972,-1.322806,1.339087,0.469766,1.163205,-1.340370,1.005749,1.008566,1.160245
164,1.029532,-0.787367,0.546355,-0.611176,0.459632,-0.841670,0.991608,0.753473,-1.355523,1.336451,0.457435,1.164562,-1.364998,1.008863,1.002613,1.155001
165,1.057198,-0.757940,0.421319,-0.599627,0.493659,-0.838453,1.002592,0.741973,-1.376175,1.333816,0.454716,1.165920,-1.389626,1.011977,0.996661,1.149757
166,1.082730,-0.917469,0.661467,-0.589237,0.617024,-1.006001,1.013577,0.730473,-1.414146,1.331180,0.446427,1.167277,-1.414254,1.015092,0.990708,1.144512


In [23]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      138.707
1      135.009
2      126.554
3      137.331
4      118.680
        ...   
163    280.074
164    277.938
165    275.802
166    273.666
167    271.530
Name: Bahia - Consumo de Cimento (t), Length: 168, dtype: float64

In [24]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [25]:
reshaped_train, reshaped_target = create_batches(train_input, train_target, 36, start_index)
reshaped_train.shape

(132, 36, 16)

In [26]:
# input para treinamento
test_input = input_data.iloc[train_split-36:split_index + 1]
test_input

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
132,0.543796,1.590448,0.225519,-0.298056,-0.311347,-0.077688,0.623805,0.982447,0.041401,0.450270,1.069363,0.900016,0.005667,0.917759,1.088312,0.982858
133,0.564912,1.702213,0.413533,-0.281023,-0.313831,-0.030826,0.635788,0.979099,-0.019620,0.450043,1.054155,0.914134,-0.050121,0.925696,1.092431,0.995759
134,0.584512,1.677438,0.431751,-0.265304,0.006739,-0.076111,0.647771,0.975750,0.029284,0.449817,1.039972,0.928253,-0.105908,0.933634,1.096551,1.008660
135,0.603507,1.638599,0.414613,-0.251648,-0.486922,-0.205002,0.659754,0.972402,0.072516,0.449591,1.025069,0.942371,-0.161695,0.941571,1.100670,1.021561
136,0.624844,1.691843,0.566348,-0.231997,-0.559925,-0.132229,0.671737,0.969054,-0.064532,0.449365,1.009687,0.956489,-0.217482,0.949509,1.104789,1.034462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.851312,-0.689886,2.909383,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628,1.041952,0.619029,0.865987,-1.416165,0.555618,0.463875,1.107985
200,1.873607,-0.441954,2.913017,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775,1.026783,0.635352,0.848481,-1.404857,0.531641,0.443334,1.106555
201,1.901718,-0.132782,2.818435,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287,1.011615,0.652040,0.830975,-1.393550,0.507664,0.422793,1.105126
202,1.928136,0.084061,2.905726,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156,0.996447,0.638794,0.813469,-1.382243,0.483687,0.402252,1.103697


In [27]:
reshaped_test, reshaped_test_target = create_batches(test_input, target_data, 36, 132)
reshaped_test.shape 

(36, 36, 16)

In [19]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * 6 + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [28]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [29]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [31]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[607932482, 3618141235, 4202845226, 3864565908, 842708788, 2521680910, 1487098388, 1305453216, 1664261261, 1702725033, 802031259, 2259736423, 929213541, 657784457, 3433843519, 967964318, 3146527401, 493776341, 1844216892, 876680700, 3963062856, 2051108872, 2450534288, 1050358393, 859958532]


Step: 0 ___________________________________________
val_loss: 154.43568420410156
winner_seed: 607932482


Step: 1 ___________________________________________
val_loss: 156.52947998046875


Step: 2 ___________________________________________
val_loss: 282.73675537109375


Step: 3 ___________________________________________
val_loss: 116.61396789550781
winner_seed: 3864565908


Step: 4 ___________________________________________
val_loss: 126.7946548461914


Step: 5 ___________________________________________
val_loss: 151.833740234375


Step: 6 ___________________________________________
val_loss: 160.74114990234375


Step: 7 ___________________________________________
val_loss: 249.00546264648438


In [32]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 111ms/step - loss: 60113.1562 - val_loss: 2966.0806
Epoch 2/10000
4/4 [==============================] - 0s 8ms/step - loss: 3716.4397 - val_loss: 3253.5015
Epoch 3/10000
4/4 [==============================] - 0s 8ms/step - loss: 3885.6323 - val_loss: 3016.9617
Epoch 4/10000
4/4 [==============================] - 0s 8ms/step - loss: 3615.0134 - val_loss: 2971.2927
Epoch 5/10000
4/4 [==============================] - 0s 8ms/step - loss: 3605.6680 - val_loss: 3241.0127
Epoch 6/10000
4/4 [==============================] - 0s 8ms/step - loss: 3711.6287 - val_loss: 3284.9827
Epoch 7/10000
4/4 [==============================] - 0s 8ms/step - loss: 3723.6125 - val_loss: 2881.4597
Epoch 8/10000
4/4 [==============================] - 0s 8ms/step - loss: 3189.0046 - val_loss: 3256.2751
Epoch 9/10000
4/4 [==============================] - 0s 8ms/step - loss: 3782.6667 - val_loss: 2947.6584
Epoch 10/10000
4/4 [==============================] 

4/4 [==============================] - 0s 8ms/step - loss: 406.3309 - val_loss: 270.8820
Epoch 80/10000
4/4 [==============================] - 0s 8ms/step - loss: 489.9063 - val_loss: 218.2407
Epoch 81/10000
4/4 [==============================] - 0s 8ms/step - loss: 579.5417 - val_loss: 438.9644
Epoch 82/10000
4/4 [==============================] - 0s 8ms/step - loss: 837.2184 - val_loss: 342.4742
Epoch 83/10000
4/4 [==============================] - 0s 8ms/step - loss: 462.1609 - val_loss: 311.8296
Epoch 84/10000
4/4 [==============================] - 0s 7ms/step - loss: 424.1825 - val_loss: 214.1479
Epoch 85/10000
4/4 [==============================] - 0s 8ms/step - loss: 384.8317 - val_loss: 260.0876
Epoch 86/10000
4/4 [==============================] - 0s 8ms/step - loss: 448.7028 - val_loss: 248.8876
Epoch 87/10000
4/4 [==============================] - 0s 8ms/step - loss: 383.9268 - val_loss: 310.4935
Epoch 88/10000
4/4 [==============================] - 0s 8ms/step - loss: 438.5

4/4 [==============================] - 0s 7ms/step - loss: 388.6042 - val_loss: 393.3872
Epoch 158/10000
4/4 [==============================] - 0s 8ms/step - loss: 505.2452 - val_loss: 338.5903
Epoch 159/10000
4/4 [==============================] - 0s 8ms/step - loss: 381.3467 - val_loss: 429.8561
Epoch 160/10000
4/4 [==============================] - 0s 7ms/step - loss: 424.9409 - val_loss: 289.8496
Epoch 161/10000
4/4 [==============================] - 0s 8ms/step - loss: 351.1176 - val_loss: 244.9048
Epoch 162/10000
4/4 [==============================] - 0s 8ms/step - loss: 374.9512 - val_loss: 310.4054
Epoch 163/10000
4/4 [==============================] - 0s 8ms/step - loss: 348.5233 - val_loss: 734.1678
Epoch 164/10000
4/4 [==============================] - 0s 8ms/step - loss: 796.9301 - val_loss: 237.7150
Epoch 165/10000
4/4 [==============================] - 0s 8ms/step - loss: 539.0856 - val_loss: 206.1410
Epoch 166/10000
4/4 [==============================] - 0s 7ms/step - lo

4/4 [==============================] - 0s 8ms/step - loss: 313.5000 - val_loss: 233.4648
Epoch 236/10000
4/4 [==============================] - 0s 7ms/step - loss: 351.4978 - val_loss: 363.9926
Epoch 237/10000
4/4 [==============================] - 0s 8ms/step - loss: 339.1375 - val_loss: 165.2144
Epoch 238/10000
4/4 [==============================] - 0s 8ms/step - loss: 294.6255 - val_loss: 493.2733
Epoch 239/10000
4/4 [==============================] - 0s 7ms/step - loss: 394.4835 - val_loss: 243.7320
Epoch 240/10000
4/4 [==============================] - 0s 8ms/step - loss: 298.6923 - val_loss: 205.9140
Epoch 241/10000
4/4 [==============================] - 0s 7ms/step - loss: 357.9200 - val_loss: 153.5442
Epoch 242/10000
4/4 [==============================] - 0s 8ms/step - loss: 315.6550 - val_loss: 264.1104
Epoch 243/10000
4/4 [==============================] - 0s 8ms/step - loss: 346.8401 - val_loss: 206.2180
Epoch 244/10000
4/4 [==============================] - 0s 7ms/step - lo

4/4 [==============================] - 0s 7ms/step - loss: 309.2505 - val_loss: 204.7279
Epoch 314/10000
4/4 [==============================] - 0s 8ms/step - loss: 323.8674 - val_loss: 218.1364
Epoch 315/10000
4/4 [==============================] - 0s 7ms/step - loss: 330.1360 - val_loss: 285.3857
Epoch 316/10000
4/4 [==============================] - 0s 7ms/step - loss: 300.5735 - val_loss: 190.3841
Epoch 317/10000
4/4 [==============================] - 0s 8ms/step - loss: 301.2668 - val_loss: 192.0186
Epoch 318/10000
4/4 [==============================] - 0s 7ms/step - loss: 310.5426 - val_loss: 374.3806
Epoch 319/10000
4/4 [==============================] - 0s 8ms/step - loss: 381.0960 - val_loss: 316.8260
Epoch 320/10000
4/4 [==============================] - 0s 8ms/step - loss: 303.1793 - val_loss: 320.5744
Epoch 321/10000
4/4 [==============================] - 0s 8ms/step - loss: 424.5437 - val_loss: 305.9799
Epoch 322/10000
4/4 [==============================] - 0s 8ms/step - lo

...

4/4 [==============================] - 0s 8ms/step - loss: 206.0060 - val_loss: 211.2536
Epoch 626/10000
4/4 [==============================] - 0s 7ms/step - loss: 191.5249 - val_loss: 398.9577
Epoch 627/10000
4/4 [==============================] - 0s 8ms/step - loss: 277.4824 - val_loss: 230.1526
Epoch 628/10000
4/4 [==============================] - 0s 8ms/step - loss: 240.1407 - val_loss: 237.5918
Epoch 629/10000
4/4 [==============================] - 0s 7ms/step - loss: 266.5890 - val_loss: 212.9523
Epoch 630/10000
4/4 [==============================] - 0s 8ms/step - loss: 246.2653 - val_loss: 522.5471
Epoch 631/10000
4/4 [==============================] - 0s 7ms/step - loss: 246.6526 - val_loss: 209.7099
Epoch 632/10000
4/4 [==============================] - 0s 7ms/step - loss: 210.6120 - val_loss: 200.1654
Epoch 633/10000
4/4 [==============================] - 0s 8ms/step - loss: 203.7906 - val_loss: 238.7152
Epoch 634/10000
4/4 [==============================] - 0s 7ms/step - lo

4/4 [==============================] - 0s 8ms/step - loss: 240.9536 - val_loss: 587.1226
Epoch 704/10000
4/4 [==============================] - 0s 7ms/step - loss: 281.2982 - val_loss: 275.4710
Epoch 705/10000
4/4 [==============================] - 0s 8ms/step - loss: 239.8183 - val_loss: 460.7037
Epoch 706/10000
4/4 [==============================] - 0s 7ms/step - loss: 246.9227 - val_loss: 320.1969
Epoch 707/10000
4/4 [==============================] - 0s 8ms/step - loss: 209.5126 - val_loss: 212.0276
Epoch 708/10000
4/4 [==============================] - 0s 8ms/step - loss: 164.5740 - val_loss: 237.2668
Epoch 709/10000
4/4 [==============================] - 0s 8ms/step - loss: 182.7396 - val_loss: 283.9581
Epoch 710/10000
4/4 [==============================] - 0s 8ms/step - loss: 245.8817 - val_loss: 225.1895
Epoch 711/10000
4/4 [==============================] - 0s 8ms/step - loss: 166.3255 - val_loss: 242.1762
Epoch 712/10000
4/4 [==============================] - 0s 7ms/step - lo

4/4 [==============================] - 0s 8ms/step - loss: 197.7488 - val_loss: 257.5193
Epoch 782/10000
4/4 [==============================] - 0s 8ms/step - loss: 169.9305 - val_loss: 247.6212
Epoch 783/10000
4/4 [==============================] - 0s 8ms/step - loss: 171.1891 - val_loss: 385.5285
Epoch 784/10000
4/4 [==============================] - 0s 7ms/step - loss: 219.4272 - val_loss: 258.2711
Epoch 785/10000
4/4 [==============================] - 0s 8ms/step - loss: 227.3425 - val_loss: 371.8365
Epoch 786/10000
4/4 [==============================] - 0s 8ms/step - loss: 212.9883 - val_loss: 268.4769
Epoch 787/10000
4/4 [==============================] - 0s 7ms/step - loss: 158.3545 - val_loss: 234.0482
Epoch 788/10000
4/4 [==============================] - 0s 8ms/step - loss: 218.8815 - val_loss: 222.4868
Epoch 789/10000
4/4 [==============================] - 0s 8ms/step - loss: 174.6586 - val_loss: 313.5175
Epoch 790/10000
4/4 [==============================] - 0s 7ms/step - lo

4/4 [==============================] - 0s 8ms/step - loss: 166.8867 - val_loss: 220.2644
Epoch 860/10000
4/4 [==============================] - 0s 8ms/step - loss: 171.9777 - val_loss: 250.9328
Epoch 861/10000
4/4 [==============================] - 0s 8ms/step - loss: 179.9562 - val_loss: 386.2065
Epoch 862/10000
4/4 [==============================] - 0s 8ms/step - loss: 150.0118 - val_loss: 240.2781
Epoch 863/10000
4/4 [==============================] - 0s 7ms/step - loss: 190.1009 - val_loss: 430.4507
Epoch 864/10000
4/4 [==============================] - 0s 8ms/step - loss: 203.3076 - val_loss: 218.0514
Epoch 865/10000
4/4 [==============================] - 0s 8ms/step - loss: 145.0450 - val_loss: 211.1822
Epoch 866/10000
4/4 [==============================] - 0s 7ms/step - loss: 161.2778 - val_loss: 653.5125
Epoch 867/10000
4/4 [==============================] - 0s 8ms/step - loss: 284.3383 - val_loss: 496.5760
Epoch 868/10000
4/4 [==============================] - 0s 7ms/step - lo

4/4 [==============================] - 0s 8ms/step - loss: 226.8716 - val_loss: 196.1912
Epoch 938/10000
4/4 [==============================] - 0s 8ms/step - loss: 292.5818 - val_loss: 223.0046
Epoch 939/10000
4/4 [==============================] - 0s 8ms/step - loss: 245.8499 - val_loss: 244.9824
Epoch 940/10000
4/4 [==============================] - 0s 8ms/step - loss: 242.0949 - val_loss: 183.9732
Epoch 941/10000
4/4 [==============================] - 0s 7ms/step - loss: 202.3073 - val_loss: 177.3823
Epoch 942/10000
4/4 [==============================] - 0s 8ms/step - loss: 219.6068 - val_loss: 294.6806
Epoch 943/10000
4/4 [==============================] - 0s 7ms/step - loss: 317.2283 - val_loss: 395.3107
Epoch 944/10000
4/4 [==============================] - 0s 8ms/step - loss: 264.3779 - val_loss: 285.1066
Epoch 945/10000
4/4 [==============================] - 0s 8ms/step - loss: 241.6789 - val_loss: 194.5084
Epoch 946/10000
4/4 [==============================] - 0s 7ms/step - lo

In [33]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    for i in range(len(test_target)):
        input_test = test_input[i:i+1]
        prediction = model.predict(input_test)
        error = np.abs(prediction - test_target[start_target + i])
        errors.append(error)
        error_percent.append(error / test_target[start_target + i])
        print(f"Month-{i + 1} - Error: {error}")
    
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [34]:
errors, mae, mape = mae_mape_calculator(trained_model, 
                                        reshaped_test, 
                                        reshaped_test_target, 0)

1/1 [==============================] - 0s 305ms/step
Month-1 - Error: [[71.347885]]
1/1 [==============================] - 0s 14ms/step
Month-2 - Error: [[0.9328613]]
1/1 [==============================] - 0s 13ms/step
Month-3 - Error: [[65.41725]]
1/1 [==============================] - 0s 14ms/step
Month-4 - Error: [[17.391327]]
1/1 [==============================] - 0s 13ms/step
Month-5 - Error: [[43.829147]]
1/1 [==============================] - 0s 14ms/step
Month-6 - Error: [[34.60875]]
1/1 [==============================] - 0s 14ms/step
Month-7 - Error: [[6.4669495]]
1/1 [==============================] - 0s 14ms/step
Month-8 - Error: [[14.292908]]
1/1 [==============================] - 0s 15ms/step
Month-9 - Error: [[6.5070496]]
1/1 [==============================] - 0s 14ms/step
Month-10 - Error: [[26.91095]]
1/1 [==============================] - 0s 14ms/step
Month-11 - Error: [[52.228867]]
1/1 [==============================] - 0s 14ms/step
Month-12 - Error: [[15.144104]]
1/1

In [35]:
display(mae)
display(mape)

27.909111

0.118665524

In [36]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        input_test = test_input[i:i+1]
        prediction = model.predict(input_test)
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [37]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 0)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[3413.3184]] - Target[3060.1059999999998]| =  Error: [[353.2124]]; MAPE:[[0.11542489]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[3318.57]] - Target[3013.2439999999997]| =  Error: [[305.32617]]; MAPE:[[0.10132807]]
1/1 [==============================] - 0s 13ms/step
Ano-11: |Prediction[[3329.085]] - Target[3254.4680000000003]| =  Error: [[74.61694]]; MAPE:[[0.02292754]]


[array([[353.2124]], dtype=float32),
 array([[305.32617]], dtype=float32),
 array([[74.61694]], dtype=float32)]

244.38518

0.0798935